In [ ]:
CREATE STORAGE INTEGRATION capstone_s3_integration
  TYPE = EXTERNAL_STAGE
  STORAGE_PROVIDER = 'S3'
  ENABLED = TRUE
  STORAGE_AWS_ROLE_ARN = 'arn:aws:iam::021891591907:role/snowflakerole_capstone_hwatari'
  STORAGE_ALLOWED_LOCATIONS = ('s3://capstone-hwatari/')